In [16]:
import pandas as pd

In [1]:
from transform_ps.etl.data_type import DataFactory
from transform_ps.etl.util import clear_cwd_cache

In [2]:
clear_cwd_cache()

cache is now empty


In [3]:
factory = DataFactory()

### Get Subscriber info

In [4]:
sub = factory.create_data_type('subscriber')

In [5]:
sub.get_data(fys=18)

Finished prepare_data in 0.0121 secs
Finished prepare_data in 0.0102 secs


In [14]:
sub.working.pkg_desc.drop_duplicates().to_excel('out.xlsx')

In [26]:
package_type = {row[0]: row[1] for index, row in pd.read_csv('out.csv').iterrows() }

In [29]:
sub.working['pkg_type'] = sub.working.pkg_desc.map(package_type)

In [35]:
s_data = sub.working[['customer_no', 'pkg_type']].drop_duplicates()

In [42]:
lookup_data = {'full': [], 'partial': []}

In [43]:
for customer, pkg_type in zip(s_data['customer_no'], s_data['pkg_type']):
    lookup_data.get(pkg_type).append(customer)

### Get Ticket Data

In [50]:
tix = factory.create_data_type('ticket')
tix.get_data(fys=[18])

not found in cache
saved working to cache
Finished prepare_data in 4.5430 secs


In [53]:
t_data = tix.type.filters['paid'](tix.working, 'paid_amt')

In [57]:
ticket_buyers = set(t_data.customer_no)

### Get Donor Info

In [6]:
donor = factory.create_data_type('donor')

In [7]:
donor.get_data(fys=13)

not found in cache
saved working to cache
Finished prepare_data in 2.3295 secs


### Filter FY18

In [8]:
d18 = donor.type.filters['fys'](donor.working, fys=[18])

### Prep Data

In [83]:
def get_category(customer_no):
    if customer_no in lookup_data.get('full'):
        return 'full'
    elif customer_no in lookup_data.get('partial'):
        return 'partial'
    elif customer_no in ticket_buyers:
        return 'single'
    else:
        return 'none'

In [84]:
donor_mapping = {'full': [], 'partial': [], 'single': [], 'none': []}

In [85]:
for customer in list(set(d18.customer_no)):
    cat = get_category(customer)
    donor_mapping.get(cat).append(customer)

##### Filter Individual Giving

In [96]:
mask = d18.campaign.str.contains('Government|Foundation|Corporate', regex=True)
gov_corp_found = d18.loc[mask].reset_index(drop=True)

### Metrics

In [94]:
len(set(d18.customer_no))

2453

In [88]:
len(donor_mapping.get('full'))

875

In [89]:
len(donor_mapping.get('partial'))

381

In [90]:
len(donor_mapping.get('single'))

354

In [91]:
len(donor_mapping.get('none'))

843

In [102]:
len(gov_corp_found.customer_no.drop_duplicates())

48